# 5.4 用 Python 發送 LINE 訊息

現在我們的程式能夠在指定的時間點自動執行。但設想一下，接下來若我們需要處理的資料量變大，意味著我們的程式可能需要運作一段時間才會執行完畢。我們不會希望在執行程式的時候，坐在電腦旁邊乾等程式執行完畢。另外，若我們的自動化程式是否有成功的執行，這個我們只能夠在程式執行完畢之後開啓檔案檢查資料才知道。此時若我們的程式能夠發送一個訊息，提醒我們程式已經執行完畢，就可以避免掉上述的狀況。

針對上述問題，LINE 即時通訊軟體有提供一個非常好用的服務，那就是 LINE Notify。

LINE Notify 是一個「訊息推播」服務 API，它的運作方式為下：

- LINE 即時通訊軟體有一個特殊的 LINE Notify 帳號，是一個專門用來發送訊息的帳號。
- 只要你有訂閱 LINE Notify 通知服務，或是把該賬號加入一個群組，此帳號就會主動發訊息給你或是群組。
- 任何一個 LINE 用戶都可以使用該服務，可以透過程式推播訊息給任何人，而且此服務完全免費。
- 而且比起傳統的 Chatbot（聊天機器人），此服務沒有複雜的驗證程序，使用者只需要申請一組權杖（其實就是通行碼）即可。

明白了 LINE Notify 服務的運作方式之後，可以發先此服務很適合與自動化程式結合，達到即時通知的目的。

在接下來的流程，我們就來見識如何利用 Python 串接 LINE Notify 服務發送訊息。

**備注：本教材由張佑成編寫，版權所有，翻印必究**

# 5.4.1 首先需要登入 Line Notify 官網

Line Notify 官網：[連結](https://notify-bot.line.me/zh_TW/)

![](https://drive.google.com/uc?export=download&id=1ud-ppeCo-L-kj1LkIDAxCb06Z2SF1XhB)

# 5.4.2 產生開發者用的權杖

登入後，進入個人頁面：

![](https://drive.google.com/uc?export=download&id=1WEWooXdro9U3bZaLIwozULRaxOR7cpgL)

在頁面最下方有一個 **發行權杖** 選項

![](https://drive.google.com/uc?export=download&id=1qrV2JZOK8Y83Y-hlQc4oHwAaDTnLHf8o)

輸入此權杖名稱，並且選擇你想發訊息的社團

![](https://drive.google.com/uc?export=download&id=1lWsK_MiwkGUW1CboViu1ZBh2Mf4GSS1m)

建議先選 1 to 1，發送訊息給自己，接下來請記下產生的權杖（紅色文字部分）

![](https://drive.google.com/uc?export=download&id=1lRYHOnFVUYr5g1QkVIG44uWVphrdsUF8)

# 5.5 用 Python 發送 Line 訊息

```python
import requests

token = "你申請到的 Line 權杖"
# 只需要把 content 的內容改成你需要傳遞的訊息即可
content = "你好,我是來自 Python 的訊息，啾咪！"
# Line Notify
line_url = "https://notify-api.line.me/api/notify"
headers = {
        "Authorization": "Bearer " + token
    }

payload = {'message': content}
r = requests.post(line_url, headers = headers, params = payload)
```


由於發送 LINE Notify 訊息的程式碼非常制式，我們將上述程式碼整理成一個函式：

```python
def notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    headers = {
            "Authorization": "Bearer " + token
        }

    payload = {"message": msg }
    r = requests.post(line_url, headers = headers, params = payload)

    return "訊息發送成功！"

line_token = "你申請到的 Line 權杖"
notify("PyXL 台股爬蟲執行完畢！", line_token)
```

在確認了函式能夠正常運作之後，我們可以在原本的爬蟲程式上加入發送 LINE 訊息通知（詳細請查看我們的完整版程式碼）

# 5.6 用 Line 發送圖片

```python
# Line Notify api 網址
url = "https://notify-api.line.me/api/notify"
# 圖片路徑
pic_url = r"C:\你的圖片路徑\python_logo.png"
# 訊息
msg = "你好,我是來自 Python 的訊息，啾咪！"

headers = {
    "Authorization": "Bearer " + token
}

# 設定訊息内容
payload = { "message": msg }
# 設定圖檔路徑
files = { "imageFile": open(pic_url, "rb") }
# 發送請求
r = requests.post(url, headers = headers, params = payload, files = files)
```

# 5.7 與我們之前的程式整合

我們可以在爬蟲程式執行完畢之後，讓電腦發送一個 Line 訊息通知我們來查詢結果：


In [2]:
import requests
from bs4 import BeautifulSoup as BS
import time
import xlwings as xw

# 輸入股票代號，回傳該股票的收盤價
def tw_stock_crawler(sid):
    res = requests.get(f"https://stock.wearn.com/cdata.asp?kind={sid}")
    res.encoding = "big5"
    html = BS(res.text, "html.parser")
    table = html.findAll("table")[0] 
    trs = table.findAll("tr")
    tr = trs[2]
    tds = tr.findAll("td")
    data = [float(td.text.replace("\xa0", "").replace(",", "")) for td in tds[1:]]
    # 以字典形式回傳我們的資料
    return {
        "open": data[0],
        "high": data[1],
        "low": data[2],
        "close": data[3],
        "volume": data[4]
    }

def notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    headers = {
            "Authorization": "Bearer " + token
        }

    payload = {"message": msg }
    r = requests.post(line_url, headers = headers, params = payload)

    return "訊息發送成功！"

wb = xw.Book("投資組合.xlsx")
date = time.strftime("%Y/%m/%d")
# 定義所有投資組合的股票代號
stocks = ["2330", "0050", "2412", "2317", "2603"]

for stock in stocks:
    # 顯示開啓工作表
    sheet.activate()
    # 截取到該股票代號的收盤價
    data = tw_stock_crawler(stock)
    # 截取相對應的工作表
    sheet = wb.sheets[stock]
    # 偵測該工作表的最後一行行數
    last_row = sheet.range("B1").end("down").row
    # 將資料寫入相對應的欄位
    sheet.range(f"A{last_row+1}").value = date
    sheet.range(f"B{last_row+1}").value = data["open"]
    sheet.range(f"C{last_row+1}").value = data["high"]
    sheet.range(f"D{last_row+1}").value = data["low"]
    sheet.range(f"E{last_row+1}").value = data["close"]
    sheet.range(f"F{last_row+1}").value = data["volume"]
    time.sleep(2)

# 儲存檔案
wb.save()
# 發送 Line 訊息
line_token = "你申請到的 Line 權杖"
notify(f"{date} PyXL 台股爬蟲執行完畢！", line_token)

延伸閲讀：

[Line Notify API 官方文件](https://developers.line.biz/en/enterprise-reference/messaging-api/?source=post_page---------------------------#sending_message)